# func

In [58]:
#!/usr/bin/env python
# coding: utf-8

# # Funções

# ## Importing libs

import os
import requests
import tweepy
from datetime import datetime as dt

# ## Auth

# tt = tweepy.Client(
#     #Consumer Keys
#     consumer_key= os.environ['CONSUMER_KEY'],
#     consumer_secret= os.environ['CONSUMER_SECRET'],
#     # Access Token and Secret
#     access_token= os.environ['ACCESS_TOKEN'],
#     access_token_secret= os.environ['ACCESS_TOKEN_SECRET'])

# ## Defining Functions

now = dt.now()
#now = dt(year=2023, month = 11, day = 15) #Data de teste! Remover!

#- get_date()
def get_date(year=None, month=None, day=None, now=now, as_str=True):
    """Returns string with the actual month and year, or especified date.
  Used in request url.

  in: get_month()
  out: <ACTUAL_YEAR>/<ACTUAL_MONTH>/

  in: get_month(year = 2010, month = 03)
  out: 2010/marco"""

    if year == None:
        year_n = now.year
    else:
        year_n = year

    if month == None:
        month_n = now.month
    else:
        month_n = month

    day_n = now.day

    dt_month = dict(
        zip(range(1, 13), [
            'janeiro', 'fevereiro', 'marco', 'abril', 'maio', 'junho', 'julho',
            'agosto', 'setembro', 'outubro', 'novembro', 'dezembro'
        ]))

    if as_str == True:
        str_date = '{}/{}'.format(year_n, dt_month[month_n])
    else:
        str_date = [year_n, month_n, day_n]

    return str_date


#- create_tree()
def create_tree(year=None, month=None, url = 'https://www.datascomemorativas.me/'):

    # sets month and year for request ()
    str_date = get_date(year, month)

    # Scrap's Resquest
    url_r = url+'{}'.format(str_date)
    with requests.get(url_r) as r:
        #print('DEBUG - status code: ',r.status_code)
        r.encoding = 'UTF-8'
        text = r.text  # string
        #content = r.content # bytes

    # generates html tree from html string
    from lxml import html
    tree = html.fromstring(text)
    return tree


#- create_grid()
def create_grid(year=None, month=None, full_grid=False):

    # today's date
    today = get_date(as_str=False)

    if year == None:
        year = today[0]

    month_n = today[1] if month == None else month

    html_tree = create_tree(year, month)

    x = list(range(1, 7))  # week of the month (1 = first week)
    y = list(range(1, 8))  # day of the week (1 = sunday, 7 = saturday)
    z = list(range(
        1, 10))  # 9 rows per day (row1: icecream day, row2: youscream day)

    # Creates Grid
    from itertools import product
    l = list(product(x, y, z))  # Vectorial product of lists

    # Sets up grid loop
    content = None
    date_list = []

    # defines year
    year_xpath = '/html/body/section/div/header/div[1]/span[2]'
    year = int(html_tree.xpath(year_xpath)[0].text_content())

    # defines month
    month_xpath = '/html/body/section/div/header/div[1]/span[1]'
    month = html_tree.xpath(month_xpath)[0].text_content()

    # Searchs html-tree using the grid
    for x in range(len(l)):
        a, b, c = l[x][0], l[x][1], l[x][2]
        xpath_ref = [a, b, c]
        # If a month starts on monday (week day 2), then the Sunday (day =1) returns Day = None
        try:
            #Defines day
            day_xpath = '/html/body/section/div/div/table/tbody/tr[{0}]/td[{1}]/div[1]/span[1]'.format(
                a, b)
            day = html_tree.xpath(day_xpath)[0].text_content()
        except:
            day = 0



        try:
            content_xpath = '/html/body/section/div/div/table/tbody/tr[{0}]/td[{1}]/ul/li[{2}]/span'.format(
                a, b, c)
            content = html_tree.xpath(content_xpath)[0].text_content()
        except:
            content = None

        # Verifica se é um feriado!
        if content == None:
            try:
                content_xpath = '/html/body/section/div/div/table/tbody/tr[{0}]/td[{1}]/ul/li[{2}]/a'.format(
                a, b, c)
                content = html_tree.xpath(content_xpath)[0].text_content()
                feriado_str = 'feriado!⭐ '
                content = feriado_str+content
            except:
                content = None           

        date_list.append([year, month_n, int(day), content, str(xpath_ref)])

        if full_grid == False:
            date_list = [x for x in date_list if x[3] != None
                         ]  # Filters grid: content != None. See *** bellow

    return date_list


#- todays_tt()
def todays_tt():
    today = get_date(as_str=False)
    date_list = create_grid()
    tt_list = [x for x in date_list if x[:3] == today]

    return tt_list


#- format_post()
def format_post():
    ttt = todays_tt()
    ttt_aux = list(set([x[3] for x in ttt]))
    txt = str([x for x in ttt_aux]).strip('[]').replace("'", "")

    if len(ttt_aux) > 1:
        txt = txt.replace(',{}'.format(txt.split(',')[-1]),
                          ' e{}'.format(txt.split(',')[-1]))
    # - Substitui texto específico
    txt = txt.replace('Dia do Deficiente Físico','Dia da Pessoa com Deficência Física')

    if txt == '':
        return 'Hoje não temos datas comemorativas! 😥'
    else:
        return 'Hoje é {}'.format(txt)


#- post_date()
def post_date():
    text = format_post()
    r = tt.create_tweet(text=text)
    print(r)
    return


# dev

- Adicionei os feriados nacionais! Atualizar código de get_gridf no github!!!

In [48]:
create_grid()

[[2023, 11, 1, 'Dia Mundial do Veganismo', '[1, 4, 1]'],
 [2023, 11, 1, 'Dia de Todos os Santos', '[1, 4, 2]'],
 [2023, 11, 2, 'feriado!⭐ Dia de Finados', '[1, 5, 1]'],
 [2023, 11, 3, 'Dia do Cabeleireiro', '[1, 6, 1]'],
 [2023, 11, 3, 'Instituição do Direito e Voto da Mulher (1930)', '[1, 6, 2]'],
 [2023, 11, 4, 'Dia do Inventor', '[1, 7, 1]'],
 [2023, 11, 5, 'Dia da Ciência e Cultura', '[2, 1, 1]'],
 [2023, 11, 5, 'Dia do Cinema Brasileiro', '[2, 1, 2]'],
 [2023, 11, 5, 'Dia do Radioamador e Técnico em Eletrônica', '[2, 1, 3]'],
 [2023, 11, 5, 'Dia Nacional do Designer', '[2, 1, 4]'],
 [2023, 11, 6, 'Dia Nacional do Amigo da Marinha do Brasil', '[2, 2, 1]'],
 [2023, 11, 6, 'Dia Nacional do Riso', '[2, 2, 2]'],
 [2023, 11, 7, 'Dia do Radialista', '[2, 3, 1]'],
 [2023, 11, 8, 'Dia do Radiologista', '[2, 4, 1]'],
 [2023, 11, 8, 'Dia Mundial do Urbanismo', '[2, 4, 2]'],
 [2023, 11, 9, 'Dia Mundial da Liberdade', '[2, 5, 1]'],
 [2023, 11, 9, 'Dia do Técnico em Eletrotécnica', '[2, 5, 2]']

In [49]:
todays_tt()

[[2023, 11, 15, 'feriado!⭐ Proclamação da República (1889)', '[3, 4, 1]']]

In [50]:
def format_post():
    ttt = todays_tt()
    ttt_aux = list(set([x[3] for x in ttt]))
    txt = str([x for x in ttt_aux]).strip('[]').replace("'", "")

    if len(ttt_aux) > 1:
        txt = txt.replace(',{}'.format(txt.split(',')[-1]),
                          ' e{}'.format(txt.split(',')[-1]))
    # - Substitui texto específico
    txt = txt.replace('Dia do Deficiente Físico','Dia da Pessoa com Deficência Física')

    if txt == '':
        return 'Hoje não temos datas comemorativas! 😥'
    else:
        return 'Hoje é {}'.format(txt)


In [51]:
format_post()

'Hoje é feriado!⭐ Proclamação da República (1889)'

# Dias até o próximo feriado

In [268]:
def create_tree_feriado(year=None, month=None, url = 'http://www.supercalendario.com.br/feriados/'):

    # sets month and year for request ()
    str_date = get_date(year, month)[:4]

    # Scrap's Resquest
    url_r = url+'{}'.format(str_date)
    #print(url_r)
    with requests.get(url_r) as r:
        #print('DEBUG - status code: ',r.status_code)
        r.encoding = 'UTF-8'
        text = r.text  # string
        #content = r.content # bytes

    # generates html tree from html string
    from lxml import html
    tree = html.fromstring(text)
    return tree

In [269]:
tree = create_tree_feriado(url = url_feriado)

In [275]:
from pandas import to_datetime

In [278]:
print(get_content_feriado(today = to_datetime(2023-12-31).date()))


Faltam 19409 dias para o próximo feriado: O Carnaval, em 21 de fevereiro de 2023!


In [279]:
print(get_content_feriado())


Faltam 32 dias para o próximo feriado: O Natal, em 25 de dezembro de 2023!
Se prepara pro feriadão que o Natal cairá numa Segunda-feira!🌞😎


In [274]:
def get_content_feriado(year = None, today = now.date()):
    
    if year == None:
        year = now.year
    
    url_feriado = 'http://www.supercalendario.com.br/feriados/'
    tree = create_tree_feriado(year = year, url = url_feriado)   

    list_contents = [y.split(' - ') for y in [x.text_content() for x in tree.find_class('holidayDate')[1:]]]
    list_feriado = [f[0] for f in list_contents]
    list_wd = [f[1] for f in list_contents]
    list_content = [f[2] for f in list_contents]

    from pandas import to_datetime
    from dateutil.relativedelta import relativedelta

    rd = (relativedelta(years = year) - relativedelta(years = 1900))  
    
    list_feriado = [(to_datetime(f, format='%d/%m') + rd).date() for f in list_feriado]

    list_distancia_feriado = [(d - today).days for d in list_feriado] # Testar d- 31/12/2023
    
    # DEBUG - TESTE!!!
    if len(list_distancia_feriado) == 0:
        get_content_feriado(year = year+1)
    
    next_holiday_in_days = min([x for x in list_distancia_feriado if (x >0)])
    next_holiday_idx = list_distancia_feriado.index(next_holiday_in_days)

    dt_month = dict(
            zip(range(1, 13), [
                'janeiro', 'fevereiro', 'marco', 'abril', 'maio', 'junho', 'julho',
                'agosto', 'setembro', 'outubro', 'novembro', 'dezembro'
            ]))

    str_dt_feriado = list_feriado[next_holiday_idx].strftime(f'%d de {dt_month[list_feriado[next_holiday_idx].month]} de %Y')
    str_content_feriado = f"\nFaltam \033[1m{list_distancia_feriado[next_holiday_idx]} dias\033[0m para o próximo feriado: O {list_content[next_holiday_idx]}, em {str_dt_feriado}!"
    extra_feriado = "\nSe prepara pro\033[1m feriadão\033[0m que o {} cairá numa {}!🌞😎".format(list_content[next_holiday_idx], list_wd[next_holiday_idx])

    if list_wd[next_holiday_idx] in ["Segunda-feira","Sexta-feira"]:
        str_content_feriado = str_content_feriado+extra_feriado
    
    #print(str_content_feriado)
    
    return str_content_feriado